<!-- Banner Image -->
<img src="https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdevnotebooks.png" width="100%">

<!-- Links -->
<center>
  <a href="https://console.brev.dev" style="color: #06b6d4;">Console</a> •
  <a href="https://brev.dev" style="color: #06b6d4;">Docs</a> •
  <a href="/" style="color: #06b6d4;">Templates</a> •
  <a href="https://discord.gg/NVDyv7TUgJ" style="color: #06b6d4;">Discord</a>
</center>

# Fine-tuning Mistral 7B using QLoRA 🤙

Welcome!

In this notebook and tutorial, we will fine-tune the [Mistral 7B](https://github.com/mistralai/mistral-src) model - which outperforms Llama 2 13B on all tested benchmarks.

This tutorial will use QLoRA, a fine-tuning method that combines quantization and LoRA. For more information about what those are and how they work, see [this post](https://brev.dev/blog/fine-tuning-mistral).

In this notebook, we will load the large model in 4bit using `bitsandbytes` (`Mistral-7B-v0.1`) and use LoRA to train using the PEFT library from Hugging Face 🤗.

Note that if you ever have trouble importing something from Huggingface, you may need to run `huggingface-cli login` in a shell. To open a shell in Jupyter Lab, click on 'Launcher' (or the '+' if it's not there) next to the notebook tab at the top of the screen. Under "Other", click "Terminal" and then run the command.

### Help us make this tutorial better! Please provide feedback on the [Discord channel]([https://discord.gg/9FPRse2K) or on [X](https://x.com/harperscarroll).

#### Before we begin: A note on OOM errors

If you get an error like this: `OutOfMemoryError: CUDA out of memory`, tweak your parameters to make the model less computationally intensive. I will help guide you through that in this guide, and if you have any additional questions you can reach out on the [Discord channel]([https://discord.gg/9FPRse2K) or on [X](https://x.com/harperscarroll).

To re-try after you tweak your parameters, open a Terminal ('Launcher' or '+' in the nav bar above -> Other -> Terminal) and run the command `nvidia-smi`. Then find the process ID `PID` under `Processes` and run the command `kill [PID]`. You will need to re-start your notebook from the beginning. (There may be a better way to do this... if so please do let me know!)

## Let's begin!
I used a GPU and dev environment from [brev.dev](https://brev.dev). Provision a pre-configured GPU in one click [here](https://console.brev.dev/environment/new?instance=g5.xlarge&name=mistral-finetune) (a single A10G or L4 should be enough for this dataset; anything with >= 24GB GPU Memory. You may need more GPUs and/or Memory if your sequence max_length is larger than 512). A few minutes after your model has started Running, click the 'Notebook' button on the top right of your screen once it illuminates (you may need to refresh the screen). You will be taken to a Jupyter Lab environment, where you can upload this Notebook.


Note: You can connect your cloud credits (AWS or GCP) by clicking "Org: " on the top right, and in the panel that slides over, click "Connect AWS" or "Connect GCP" under "Connect your cloud" and follow the instructions linked to attach your credentials.



In [1]:
# # You only need to run this once per machine
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U datasets scipy ipywidgets

### 0. Accelerator

Set up the Accelerator. I'm not sure if we really need this for a QLoRA given its [description](https://huggingface.co/docs/accelerate/v0.19.0/en/usage_guides/fsdp) (I have to read more about it) but it seems it can't hurt, and it's helpful to have the code for future reference. You can always comment out the accelerator if you want to try without.

In [2]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

### 1. Load Dataset

Let's load a meaning representation dataset, and fine-tune Mistral on that. This is a great fine-tuning dataset as it teaches the model a unique form of desired output on which the base model performs poorly out-of-the box, so it's helpful to easily and inexpensively gauge whether the fine-tuned model has learned well. (Sources: [here](https://ragntune.com/blog/gpt3.5-vs-llama2-finetuning) and [here](https://www.anyscale.com/blog/fine-tuning-is-for-form-not-facts)) (In contrast, if you fine-tune on a fact-based dataset, the model may already do quite well on that, and gauging learning is less obvious / may be more computationally expensive.)

In [3]:
# from datasets import load_dataset

# train_dataset = load_dataset('gem/viggo', split='train')
# eval_dataset = load_dataset('gem/viggo', split='validation')
# test_dataset = load_dataset('gem/viggo', split='test')

In [4]:
from datasets import load_dataset
train_dataset = load_dataset("Babelscape/REDFM", language='fr', split='train')
eval_dataset = load_dataset("Babelscape/REDFM", language='fr', split='validation')
test_dataset = load_dataset("Babelscape/REDFM", language='fr', split='test')

In [5]:
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['docid', 'title', 'uri', 'text', 'entities', 'relations'],
    num_rows: 1865
})
Dataset({
    features: ['docid', 'title', 'uri', 'text', 'entities', 'relations'],
    num_rows: 416
})
Dataset({
    features: ['docid', 'title', 'uri', 'text', 'entities', 'relations'],
    num_rows: 415
})


### 2. Load Base Model

Let's now load Mistral - `mistralai/Mistral-7B-v0.1` - using 4-bit quantization!

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
# base_model_id = "bofenghuang/vigostral-7b-chat"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/home/xli/miniconda3/envs/llm_venv/lib/python3.10/site-packages/bitsandbytes-0.41.1-py3.10.egg/bitsandbytes/cuda_setup/main.py:106: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=116


  warn((f'\n\n{"="*80}\n'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

In [8]:
def get_relation(example):
    RELATION_NAMES=['pays', 'lieu de naissance', 'conjoint', 'pays de nationalité', 'instance de',
            'capital', 'enfant', 'partage la frontière avec', 'auteur', 'directeur', 'occupation',
              'fondée par', 'ligue', 'appartenant à', 'genre', 'nommé d\'après', 'suit',
                'localisation du siège social', 'membre du casting', 'constructeur',
                  'situé dans ou à côté d\'une étendue d\'eau', 'localisation', 'partie de', 
                  'embouchure du cours d\'eau', 'membre de', 'sport', 'caractères',
                    'participant', 'travail remarquable', 'remplacer', 'frère et sœur', 'création']
    relations = []
    for relation in example['relations']:
        object = relation['object']['surfaceform']
        subject = relation['subject']['surfaceform']
        predicate = RELATION_NAMES[relation['predicate']]
        relations.append(f"[’{subject}’, ’{predicate}’, ’{object}’]")

 
    return ' | '.join(relations)

### 3. Tokenization

Set up the tokenizer. Add padding on the left as it [makes training use less memory](https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa).


In [9]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.model_max_length

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1000000000000000019884624838656

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setup the tokenize function to make labels and input_ids the same. This is basically what [self-supervised fine-tuning is](https://neptune.ai/blog/self-supervised-learning):

In [10]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

And convert each sample into a prompt that I found from [this notebook](https://github.com/samlhuillier/viggo-finetune/blob/main/llama/fine-tune-code-llama.ipynb).

In [11]:
def generate_and_tokenize_prompt(example):
    full_prompt =f"""Vous êtes un expert en data science et en traitement du langage naturel(NLP). Votre tâche consiste à extraire les triplets du TEXTE fourni ci-dessous. Un triplet de connaissances est constitué de 2 entités (sujet et objet) liées par un prédicat : ['sujet', 'prédicat', 'objet']. Les triples multiples doivent être séparés par ' | '.\n

### TEXTE:
{example["text"]}

### Relations:
{get_relation(example)}
"""
    return tokenize(full_prompt)
    # return full_prompt

Reformat the prompt and tokenize each sample:

In [12]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/1865 [00:00<?, ? examples/s]

Map:   0%|          | 0/416 [00:00<?, ? examples/s]

Check that `input_ids` is padded on the left with the `eos_token` (2) and there is an `eos_token` 2 added to the end, and the prompt starts with a `bos_token` (1).


In [13]:
print(tokenized_train_dataset[4]['input_ids'])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 550, 607, 28705, 28876, 2097, 521, 7583, 481, 1178, 6691, 911, 481, 10610, 1116, 1415, 9848, 465, 4735, 28714, 28732, 28759, 11661, 609, 550,

Check that a sample has the max length, i.e. 512.

In [14]:
print(len(tokenized_train_dataset[4]['input_ids']))

512


#### How does the base model do?

Let's grab a test input (`meaning_representation`) and desired output (`target`) pair to see how the base model does on it.

In [8]:
example = test_dataset[1]
print("Texte: " + example['text'])
print("Relations: " + get_relation(example) + "\n")

Texte: Chang'e 4 (du , de "Chang'e", déesse de la Lune dans la mythologie chinoise) est une sonde spatiale lunaire chinoise dont le lancement a eu lieu le . L'engin est une réplique de la sonde lunaire Chang'e 3, lancée en 2013. C'est le engin spatial chinois lancé vers la Lune et le deuxième à s'y poser. Chang'e 4 comprend un atterrisseur et un rover. Les deux engins spatiaux emportent plusieurs instruments dont des caméras, un spectromètre infrarouge pour mesurer la composition du sol à proximité du rover et un radar détectant la structure superficielle du sous-sol ainsi qu'un spectromètre radio pour analyser les éruptions solaires. La mission primaire doit durer 90 jours. 
Relations: [’Chang'e 4’, ’nommé d'après’, ’Chang'e’] | [’Chang'e 4’, ’instance de’, ’sonde spatiale’] | [’Chang'e 4’, ’suit’, ’Chang'e 3’]



In [9]:
eval_prompt = f"""Vous êtes un expert en data science et en traitement du langage naturel(NLP). Votre tâche consiste à extraire les triplets du TEXTE fourni ci-dessous. Un triplet de connaissances est constitué de 2 entités (sujet et objet) liées par un prédicat : ['sujet', 'prédicat', 'objet']. Les triples multiples doivent être séparés par ' | '.\n

### Texte :
{example['text']}

### Relations :
"""

In [10]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# model.eval()
# with torch.no_grad():
#     print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

We can see it doesn't do very well out of the box.

### 4. Set Up LoRA

Now, to start our fine-tuning, we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [15]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [16]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [21]:
print_trainable_parameters(model)

trainable params: 0 || all params: 3752071168 || trainable%: 0.0


Let's print the model to examine its layers, as we will apply QLoRA to all the linear layers of the model. Those layers are `q_proj`, `k_proj`, `v_proj`, `o_proj`, `gate_proj`, `up_proj`, `down_proj`, and `lm_head`.

In [22]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

Here we define the LoRA config.

`r` is the rank of the low-rank matrix used in the adapters, which thus controls the number of parameters trained. A higher rank will allow for more expressivity, but there is a compute tradeoff.

`alpha` is the scaling factor for the learned weights. The weight matrix is scaled by `alpha/r`, and thus a higher value for `alpha` assigns more weight to the LoRA activations.

The values used in the QLoRA paper were `r=64` and `lora_alpha=16`, and these are said to generalize well, but we will use `r=8` and `lora_alpha=16` so that we have more emphasis on the new fine-tuned data while also reducing computational complexity.

In [17]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)

trainable params: 21260288 || all params: 3773331456 || trainable%: 0.5634354746703705


See how the model looks different now, with the LoRA adapters added:

In [24]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=1024, bias=False
       


Let's use Weights & Biases to track our training metrics. You'll need to apply an API key when prompted. Feel free to skip this if you'd like, and just comment out the `wandb` parameters in the `Trainer` definition below.

In [1]:
import os, wandb
# os.environ["WANDB_PROJECT"] = "digital_safety"
os.environ["WANDB_BASE_URL"]="https://api.wandb.ai"
wandb.init(project="digital_safety", entity="xianli")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xianli. Use `wandb login --relogin` to force relogin


### 5. Run Training!

I used 500 steps, but I found the model should have trained for longer as it had not converged by then, so I upped the steps to 1000 below.

A note on training. You can set the `max_steps` to be high initially, and examine at what step your model's performance starts to degrade. There is where you'll find a sweet spot for how many steps to perform. For example, say you start with 1000 steps, and find that at around 500 steps the model starts overfitting - the validation loss goes up (bad) while the training loss goes down significantly, meaning the model is learning the training set really well, but is unable to generalize to new datapoints. Therefore, 500 steps would be your sweet spot, so you would use the `checkpoint-500` model repo in your output dir (`mistral-finetune-viggo`) as your final model in step 6 below.

You can interrupt the process via Kernel -> Interrupt Kernel in the top nav bar once you realize you didn't need to train anymore.

In [25]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [27]:
import transformers
from datetime import datetime

project = "KG-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "../../models/" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=8,
        max_steps=1000,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=50,
        bf16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 126.00 MiB (GPU 0; 8.00 GiB total capacity; 6.96 GiB already allocated; 0 bytes free; 7.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### 6. Drum Roll... Try the Trained Model!

By default, the PEFT library will only save the QLoRA adapters, so we need to first load the base Mistral model from the Huggingface Hub:


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# base_model_id = "bofenghuang/vigo-mistral-7b-chat"
base_model_id = "bofenghuang/vigostral-7b-chat"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/home/xli/miniconda3/envs/llm_venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/xli/miniconda3/envs/llm_venv/lib/python3.10/site-packages/bitsandbytes-0.41.1-py3.10.egg/bitsandbytes/cuda_setup/main.py:106: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=116


  warn((f'\n\n{"="*80}\n'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/xli/miniconda3/envs/llm_venv/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Now load the QLoRA adapter from the appropriate checkpoint directory, i.e. the best performing model checkpoint:

In [2]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "../../models/mistral-KG-finetune/checkpoint-200")

and run your inference!

Let's try the same `eval_prompt` and thus `model_input` as above, and see if the new finetuned model performs better.

In [11]:
ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100, pad_token_id=2)[0], skip_special_tokens=True))

Vous êtes un expert en data science et en traitement du langage naturel(NLP). Votre tâche consiste à extraire les triplets du TEXTE fourni ci-dessous. Un triplet de connaissances est constitué de 2 entités (sujet et objet) liées par un prédicat : ['sujet', 'prédicat', 'objet']. Les triples multiples doivent être séparés par ' | '.


### Texte :
Chang'e 4 (du , de "Chang'e", déesse de la Lune dans la mythologie chinoise) est une sonde spatiale lunaire chinoise dont le lancement a eu lieu le . L'engin est une réplique de la sonde lunaire Chang'e 3, lancée en 2013. C'est le engin spatial chinois lancé vers la Lune et le deuxième à s'y poser. Chang'e 4 comprend un atterrisseur et un rover. Les deux engins spatiaux emportent plusieurs instruments dont des caméras, un spectromètre infrarouge pour mesurer la composition du sol à proximité du rover et un radar détectant la structure superficielle du sous-sol ainsi qu'un spectromètre radio pour analyser les éruptions solaires. La mission primai

In [4]:
example = test_dataset[1]
print("Texte: " + example['text'])
print("Relations: " + get_relation(example) + "\n")

Texte: Chang'e 4 (du , de "Chang'e", déesse de la Lune dans la mythologie chinoise) est une sonde spatiale lunaire chinoise dont le lancement a eu lieu le . L'engin est une réplique de la sonde lunaire Chang'e 3, lancée en 2013. C'est le engin spatial chinois lancé vers la Lune et le deuxième à s'y poser. Chang'e 4 comprend un atterrisseur et un rover. Les deux engins spatiaux emportent plusieurs instruments dont des caméras, un spectromètre infrarouge pour mesurer la composition du sol à proximité du rover et un radar détectant la structure superficielle du sous-sol ainsi qu'un spectromètre radio pour analyser les éruptions solaires. La mission primaire doit durer 90 jours. 
Relations: [’Chang'e 4’, ’nommé d'après’, ’Chang'e’] | [’Chang'e 4’, ’instance de’, ’sonde spatiale’] | [’Chang'e 4’, ’suit’, ’Chang'e 3’]



In [ ]:
tokenizer = BertTokenizer.from_pretrained('path/to/vocab.txt',local_files_only=True)
model = BertForMaskedLM.from_pretrained('/path/to/pytorch_model.bin',config='../config.json', local_files_only=True)

In [1]:
import torch
from peft import PeftModel 
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# base_model_id = "bofenghuang/vigo-mistral-7b-chat"
# base_model_id = "bofenghuang/vigostral-7b-chat"
base_model_id = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/home/xli/miniconda3/envs/llm_venv/lib/python3.10/site-packages/bitsandbytes-0.41.1-py3.10.egg/bitsandbytes/cuda_setup/main.py:106: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=116


  warn((f'\n\n{"="*80}\n'
/home/xli/miniconda3/envs/llm_venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/xli/miniconda3/envs/llm_venv/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
adapter_model_id =  "../../models/merged_model"
model = PeftModel.from_pretrained(base_model, adapter_model_id)

In [4]:
from transformers import pipeline

pipeline_peft = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            # torch_dtype=torch.bfloat16,
            # generation_config=gen_cfg,
            temperature=0.1,
            trust_remote_code=True,
            device_map="auto",
            return_full_text=False,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', 'Prophe

In [14]:
from transformers import GenerationConfig
def generate_text(query):
    gen_cfg = GenerationConfig.from_model_config(model.config)
    # gen_cfg.max_new_tokens = DEFAULT_MAX_LENGTH
    # gen_cfg.temperature = 0.51
    gen_cfg.num_return_sequences = 1
    gen_cfg.use_cache = True
    gen_cfg.min_length = 1

    inputs = tokenizer(query, 
                       return_tensors="pt", 
                       return_attention_mask=False)#.to('cuda')
    outputs = model.generate(
        **inputs,
        max_length=1000, 
        generation_config=gen_cfg,
        do_sample=True, 
        temperature=0.1, 
        top_p=0.9, 
        # use_cache=True, 
        repetition_penalty=1.2, 
        eos_token_id=tokenizer.eos_token_id)
    text = tokenizer.batch_decode(outputs)[0]
    return text

In [10]:
example = test_dataset[1]
print("Texte: " + example['text'])
print("Relations: " + get_relation(example) + "\n")

eval_prompt = f"""Vous êtes un expert en data science et en traitement du langage naturel(NLP). Votre tâche consiste à extraire les triplets du TEXTE fourni ci-dessous. Un triplet de connaissances est constitué de 2 entités (sujet et objet) liées par un prédicat : ['sujet', 'prédicat', 'objet']. Les triples multiples doivent être séparés par ' | '.\n

### Texte :
{example['text']}

### Relations :
"""

Texte: Chang'e 4 (du , de "Chang'e", déesse de la Lune dans la mythologie chinoise) est une sonde spatiale lunaire chinoise dont le lancement a eu lieu le . L'engin est une réplique de la sonde lunaire Chang'e 3, lancée en 2013. C'est le engin spatial chinois lancé vers la Lune et le deuxième à s'y poser. Chang'e 4 comprend un atterrisseur et un rover. Les deux engins spatiaux emportent plusieurs instruments dont des caméras, un spectromètre infrarouge pour mesurer la composition du sol à proximité du rover et un radar détectant la structure superficielle du sous-sol ainsi qu'un spectromètre radio pour analyser les éruptions solaires. La mission primaire doit durer 90 jours. 
Relations: [’Chang'e 4’, ’nommé d'après’, ’Chang'e’] | [’Chang'e 4’, ’instance de’, ’sonde spatiale’] | [’Chang'e 4’, ’suit’, ’Chang'e 3’]



In [15]:
resp = generate_text(eval_prompt)

print(resp)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Vous êtes un expert en data science et en traitement du langage naturel(NLP). Votre tâche consiste à extraire les triplets du TEXTE fourni ci-dessous. Un triplet de connaissances est constitué de 2 entités (sujet et objet) liées par un prédicat : ['sujet', 'prédicat', 'objet']. Les triples multiples doivent être séparés par ' | '.


### Texte :
Chang'e 4 (du , de "Chang'e", déesse de la Lune dans la mythologie chinoise) est une sonde spatiale lunaire chinoise dont le lancement a eu lieu le . L'engin est une réplique de la sonde lunaire Chang'e 3, lancée en 2013. C'est le engin spatial chinois lancé vers la Lune et le deuxième à s'y poser. Chang'e 4 comprend un atterrisseur et un rover. Les deux engins spatiaux emportent plusieurs instruments dont des caméras, un spectromètre infrarouge pour mesurer la composition du sol à proximité du rover et un radar détectant la structure superficielle du sous-sol ainsi qu'un spectromètre radio pour analyser les éruptions solaires. La mission pr

### Sweet... it worked! The fine-tuned model now understands the meaning representation!

I hope you enjoyed this tutorial on fine-tuning Mistral. If you have any questions, feel free to reach out to me on [X](https://x.com/harperscarroll) or on the [Discord channel](https://discord.gg/9FPRse2K).

🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙